This notebook shows how to run a prediction on a trained assembled resnet model. 

In [ ]:
%cd ../assembled-cnn

In [3]:
import sys
sys.path.append('.')

In [4]:
import tensorflow as tf
tf.__version__

/home/jwf5/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jwf5/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jwf5/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jwf5/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: P

'1.14.0'

# Import

In [21]:
from functions import model_fns
from preprocessing import imagenet_preprocessing
from utils.data_util import *
from utils.hook_utils import *
from metric import ece_metric
from nets import hparams_config
from nets.optimizer_setting import *
from official.utils.flags import core as flags_core
from official.utils.logs import hooks_helper
from official.utils.logs import logger
from official.utils.misc import model_helpers
from utils import checkpoint_utils
from utils import data_util
from utils import export_utils
from utils.hook_utils import *
from utils import config_utils
from functions import data_config, model_fns, input_fns
from nets import blocks
from losses import cls_losses
from functions import input_fns, data_config, model_fns
from nets import run_loop_classification
from official.utils.flags import core as flags_core
from official.utils.logs import logger
from utils import log_utils
from utils import config_utils
from absl import app as absl_app
from metric import recall_metric
from absl import flags
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [6]:
def define_flags():
  run_loop_classification.define_resnet_flags(
    resnet_size_choices=['18', '34', '50', '101', '152', '200'])
  flags.adopt_module_key_flags(run_loop_classification)
  flags_core.set_defaults(train_epochs=1)
  tf.app.flags.DEFINE_string('f', '', 'kernel')
define_flags()

In [ ]:
import logging
def define_log_level():
  tf_logger = logging.getLogger('tensorflow')
  tf_logger.propagate = False
  handler = logging.StreamHandler(sys.stdout)
  formatter = logging.Formatter("%(asctime)s.%(msecs).3d %(levelname).1s: %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
  handler.setFormatter(formatter)
  tf_logger.handlers = [handler]
  tf_logger.setLevel(tf.logging.INFO)
define_log_level()

In [23]:
flags_obj = tf.app.flags.FLAGS

# Setting Hyperparameters





In [7]:
flags_obj.preprocessing_type='imagenet_224_256'
flags_obj.dataset_name='iFood2019' 
flags_obj.model_dir='train/m2' 
flags_obj.pretrained_model_checkpoint_path='train/m2'
flags_obj.num_gpus=1 
flags_obj.resnet_version='2' 
flags_obj.use_sk_block=True 
flags_obj.mixup_type=1 
flags_obj.autoaugment_type=None
flags_obj.resnet_size=50
flags_obj.use_dropblock=True 
flags_obj.dropblock_kp=[0.9,0.7]
flags_obj.batch_size=64 
flags_obj.base_learning_rate=0.0025
flags_obj.learning_rate_decay_type='cosine' 
flags_obj.train_epochs=1
flags_obj.weight_decay=0.001 
flags_obj.lr_warmup_epochs=1
flags_obj.epochs_between_evals=5
flags_obj.save_checkpoints_epochs=5
flags_obj.keep_checkpoint_max = 5
flags_obj.dtype = 'fp16'
flags_obj.export_dir = 'train/m2/export_test'
flags_obj.data_format = 'channels_last'
flags_obj.export_only = True

NameError: name 'flags_obj' is not defined

In [25]:
! ls 

Assemble-ResNet50		      NOTICE
Assemble-ResNet50.zip		      official
datasets			      output.txt
events.out.tfevents.1595374229.ml-21  preprocessing
events.out.tfevents.1595375729.ml-21  README.md
figures				      saved_model.pb
functions			      scripts
kd				      serve
LICENSE				      test
losses				      tf_summary
__MACOSX			      the_model
main_classification.py		      the_model_test.pbtxt
mce				      train
metric				      utils
nets


In [26]:
flags_obj.train_regex = '*train_TFR*' # train + validation
flags_obj.val_regex = '*val_TFR*'
flags_obj.data_dir = '../FOODX-251_Dataset'

# Define Input Function

In [27]:
from official.utils.misc import distribution_utils
def parse_record_tf_dataset(raw_record, is_training, num_channels, dtype,
                            use_random_crop=True, image_size=224,
                            autoaugment_type=None, with_drawing_bbox=False,
                            dct_method="", preprocessing_type='imagenet',
                            return_logits=False, num_classes=1001, return_filename=False,
                            use_color_jitter=False):
  feature_def = {'label': tf.FixedLenFeature([], dtype=tf.int64, default_value=-1),
                 'image': tf.FixedLenFeature([], dtype=tf.string, default_value='')}
  features = tf.io.parse_single_example(serialized=raw_record, features=feature_def)

  sup_image = preprocess_image(image_buffer=features['image'],
                               is_training=is_training,
                               num_channels=num_channels,
                               dtype=dtype,
                               use_random_crop=use_random_crop,
                               image_size=image_size,
                               autoaugment_type=autoaugment_type,
                               dct_method=dct_method,
                               decoder_name='auto',
                               preprocessing_type=preprocessing_type)
  results = {}
  results['image'] = sup_image
  results['label'] = features['label']
  return results
def input_fn_tfds(is_training, use_random_crop, num_epochs, flags_obj):
  if flags_obj.mixup_type == 1 and is_training:
    batch_size = flags_obj.batch_size * 2
    num_epochs = num_epochs * 2
  else:
    batch_size = flags_obj.batch_size

  batch_size = distribution_utils.per_device_batch_size(batch_size, flags_core.get_num_gpus(flags_obj))
  
  filenames_sup = data_util.get_filenames(is_training, flags_obj.data_dir,
                                          train_regex=flags_obj.train_regex,
                                          val_regex=flags_obj.val_regex)
  tf.logging.info('The # of Supervised tfrecords: {}'.format(len(filenames_sup)))
  dataset_meta = data_config.get_config(flags_obj.dataset_name)
  datasets = []
  dataset_sup = input_fns.input_fn(is_training,
                         filenames_sup,
                         use_random_crop,
                         batch_size,
                         dataset_meta.num_train_files,
                         dataset_meta.num_images['train'],
                         dataset_meta.shuffle_buffer,
                         dataset_meta.num_channels,
                         num_epochs,
                         flags_core.get_num_gpus(flags_obj),
                         flags_core.get_tf_dtype(flags_obj),
                         autoaugment_type=flags_obj.autoaugment_type,
                         with_drawing_bbox=flags_obj.with_drawing_bbox,
                         drop_remainder=False,
                         preprocessing_type=flags_obj.preprocessing_type,
                         return_logits=flags_obj.kd_temp > 0,
                         dct_method=flags_obj.dct_method,
                         parse_record_fn=parse_record_tf_dataset)
  datasets.append(dataset_sup)

  def flatten_input(*features):
    images_dict = {}
    for feature in features:
      for key in feature:
        if key == 'label':
          label = feature[key]
        else:
          images_dict[key] = feature[key]
    return images_dict, label

  dataset = tf.data.Dataset.zip(tuple(datasets))
  dataset = dataset.map(flatten_input)
  tf.logging.info('dataset = dataset.map(flatten_input)')
  tf.logging.info(dataset)
  return dataset


In [28]:
dataset = data_config.get_config(flags_obj.dataset_name)

In [ ]:

run_loop_classification.resnet_main(
    flags_obj, model_fns.model_fn_cls, input_fn_tfds, dataset.dataset_name,
    shape=[dataset.default_image_size, dataset.default_image_size, dataset.num_channels],
    num_images=dataset.num_images, zeroshot_eval=flags_obj.zeroshot_eval)

In [ ]:
from nets import run_loop_classification_predict
from functions import model_fns_predict
flags_obj.eval_only = True
flags_obj.pretrained_model_checkpoint_path = flags_obj.model_dir
dataset = data_config.get_config(flags_obj.dataset_name)
hist = run_loop_classification_predict.resnet_main(
    flags_obj, model_fns_predict.model_fn_cls, input_fn_tfds, dataset.dataset_name,
    shape=[dataset.default_image_size, dataset.default_image_size, dataset.num_channels],
    num_images=dataset.num_images, zeroshot_eval=flags_obj.zeroshot_eval)

In [ ]:
# for pred_dict in hist:
#     print(pred_dict['probabilities'])
# print(hist[0]['class_ids'])
import numpy as np
results = []
count = 0
for pred_dict in hist:
    results.append(pred_dict['probabilities'])
    count+=1
print(count)
# print(pred_dict['probabilities'])

In [ ]:
resultsMaxIndex = []

for i in results:
    resultsMaxIndex.append(np.argmax(i))


In [ ]:
import pandas as pd
#Macornutrient data
macro_df = pd.read_csv('../FOODX-251_Dataset/macro_data_foodX251.csv')
dataframe=pd.DataFrame(resultsMaxIndex, columns=['label']) 
predictions1 = dataframe.reset_index().merge(macro_df, how='left', on='label', sort=False).sort_index()
predictions1.head()

In [ ]:
test_df = pd.read_csv('../FOODX-251_Dataset/val_labels.csv')
# m_test_df = pd.merge(test_df, macro_df, on='label')


In [ ]:
m_test_df = test_df.merge(test_df.merge(macro_df, how='left', on='label', sort=False))
m_test_df.head()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
RSMEArray = []
RSMEArray.append( np.sqrt(mean_squared_error(m_test_df['Energy'], predictions1['Energy'])))
RSMEArray.append( np.sqrt(mean_squared_error(m_test_df['Protein'],predictions1['Protein'])))
RSMEArray.append( np.sqrt(mean_squared_error(m_test_df['Fat'], predictions1['Fat'])))
RSMEArray.append( np.sqrt(mean_squared_error(m_test_df['Carbohydrate'], predictions1['Carbohydrate'])))
print("RSME for Energy: "  + str(RSMEArray[0]))
print("RSME for Protein: "  + str(RSMEArray[1])) 
print("RSME for Fat: "  + str(RSMEArray[2]))
print("RSME for Carbohydrate: "  + str(RSMEArray[3]))
print()
print()

In [ ]:
predTotalE = []
predTotalP = []
predTotalF = []
predTotalC = []
for j in results:
    count = 0
    sumEnergy = 0
    sumProtein = 0
    sumFat = 0
    sumCarb = 0
    for i in j:
        sumEnergy += macro_df['Energy'].iloc[count] * i
        sumProtein += macro_df['Protein'].iloc[count] * i
        sumFat += macro_df['Fat'].iloc[count] * i
        sumCarb += macro_df['Carbohydrate'].iloc[count] * i
        count+=1
    predTotalE.append(sumEnergy)
    predTotalP.append(sumProtein)
    predTotalF.append(sumFat)
    predTotalC.append(sumCarb)
predictions1=pd.DataFrame({'Energy':predTotalE, 'Protein':predTotalP, 'Fat':predTotalF, 'Carbohydrate':predTotalC}) 
predictions1.head()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
RSMEArray = []
RSMEArray.append( np.sqrt(mean_squared_error(m_test_df['Energy'], predictions1['Energy'])))
RSMEArray.append( np.sqrt(mean_squared_error(m_test_df['Protein'],predictions1['Protein'])))
RSMEArray.append( np.sqrt(mean_squared_error(m_test_df['Fat'], predictions1['Fat'])))
RSMEArray.append( np.sqrt(mean_squared_error(m_test_df['Carbohydrate'], predictions1['Carbohydrate'])))
print("RSME for Energy: "  + str(RSMEArray[0]))
print("RSME for Protein: "  + str(RSMEArray[1])) 
print("RSME for Fat: "  + str(RSMEArray[2]))
print("RSME for Carbohydrate: "  + str(RSMEArray[3]))
print()
print()

In [ ]:
sumUp = 0
count = 0
for new in predictions1['Energy']:
    sumUp = sumUp + abs((m_test_df['Energy'][count] - new) / m_test_df['Energy'][count])
    count += 1
print((sumUp/len(predictions1))*100)
    